In [ ]:
from __future__ import division
%matplotlib inline
from datetime import datetime
import glob
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = (8.0, 8.0)
import pkg_resources
import time

import CameraNetwork
import CameraNetwork.global_settings as gs
import CameraNetwork.sunphotometer as spm

In [ ]:
client = CameraNetwork.CLIclient()
proxy_params = CameraNetwork.retrieve_proxy_parameters()
client.start(proxy_params)
time.sleep(3)
SERVERS = client.client_instance.servers

## Set of servers and date

In [ ]:
SERVERS = ["109", "110", "111", "112", "113", "114", "115", "116"]
extrinsic_day = datetime(2017, 3, 5)
radiometric_day = datetime(2017, 3, 5)

## Do extrinsic calibration for the specific day

In [ ]:
results = client.extrinsic(SERVERS, extrinsic_day, 0.05, True)

## Do radiometric calibration for the specific day

In [ ]:
base_path = pkg_resources.resource_filename(
    'CameraNetwork',
    '../data/aeronet/{}/*.alm'.format(radiometric_day.strftime("%Y_%m"))
)
path = glob.glob(base_path)
df = spm.parseSunPhotoMeter(path[0])
spm_df = df[radiometric_day.strftime("%Y-%m-%d")]
spm_df = [spm_df[spm_df["Wavelength(um)"]==wl] for wl in (0.6744, 0.5000, 0.4405)]
items = [d.strftime("%H:%M:%S") for d in spm_df[0].index]
items

In [ ]:
results = client.radiometric(SERVERS, radiometric_day, 4, 1, True)

In [ ]:
f, ax = plt.subplots(ncols=3, nrows=len(SERVERS), figsize=(9, len(SERVERS)*3))
for i, server_id in enumerate(SERVERS):
    angles = results[1][i]["angles"]
    estimations = results[1][i]["estimations"]
    measurements = results[1][i]["measurements"]
    ratios = results[1][i]["ratios"]
    print server_id, ratios
    for j in range(3):
        ax[i, j].plot(angles, estimations[j], label="estim")
        ax[i, j].plot(angles, measurements[j], label="meas")
        ax[i, j].legend()
        ax[i, j].set_title(server_id)

## Get images of clear sky

In [ ]:
results = client.query(SERVERS, radiometric_day)

In [ ]:
results = client.seek(SERVERS, "2017-03-05 14:05:00", hdr_index="2", jpeg=False, resolution=301)

In [ ]:
for img in results[0]:
    plt.figure()
    img = np.clip(img, 0, 50)
    plt.imshow((img/img.max()*255).astype(np.uint8), origin="bottom")